In [ ]:

def clickAnalyzer(inputFileFolder,outputFileFolder,userSessionFile,sessionOrCluster,cluster='None',app_name = 'ORA_FSCM_UIAPP',interestedClick='None',regionViewId='None',
                  delta=3,clickidPattern='None',userFilter='None',highlightEdgeCount=2,
                  date='None',totalTimeThreshold='None'):
        import os
        import glob
        import pandas as pd
        import time
        start_time = time.time()

        # Code to read all log files
        log=pd.DataFrame()
        
        
        if inputFileFolder.endswith(".log") == True:
            log = pd.read_csv(inputFileFolder,sep=r'\\t', engine='python',header=None,names='a')
        else:
            path =inputFileFolder
            filenames = glob.glob(path + "/*.log")
            for filename in filenames:
                print("Log Files got : ", filename)
                if os.stat(filename).st_size != 0 :
                    f = pd.read_csv(filename, sep=r'\\t', engine='python', header=None, names='a')                
                    log = log.append(f, ignore_index=True)
        
        if (len(log) == 0):
            print("Given input dir is empty or no .log files present. Exiting")
            return
        else:
            clickFile = log
            print("Total lines in consolidated log file to be processed, before any filters : ", log.shape)
        
        specificSession = 'None'
        cluster = 'None'
        
        l1 = len(str(sessionOrCluster))
        if l1 == 0:
            return
        else:
            if (l1 ==1) or (l1==2) or (l1==3):
                cluster = int(sessionOrCluster)
                print("Cluster chosen : ", cluster)
            elif l1 > 2:
                cluster = 'None'
                specificSession = str(sessionOrCluster)
                s_list = specificSession.split("^")
                specificSession = '\^'.join(s_list)
                print("Specific session chosen : ",specificSession )
        
#        cd $logFilePath

        import re
        import numpy as np
        from graphviz import Digraph
        from collections import defaultdict
        import math

        output_graph_name = 'ClickGraph' #outputFile[outputFile.rindex("\\")+1:outputFile.rfind(".")]+"-graph.jpeg"
        
        # Code to read all log files
        l=clickFile[clickFile.a.str.contains(app_name)]
        #l = pd.read_csv(clickFile, sep=r'\\t', engine='python', header=None, names='a')
        o = pd.DataFrame()
        o = pd.read_csv(userSessionFile)
        
        #For Specific session
        if specificSession != 'None':
            withspecificSession = l[l.a.str.contains(specificSession)]
            if (len(withspecificSession) < 1 ):
                print("Chosen Session is not present, exiting !")
                return
        else:
            withspecificSession = l
            
        #change1 withChcid = withspecificSession[withspecificSession.a.str.contains("CH_CID")]
        
        
        withChcnm = withspecificSession[withspecificSession.a.str.contains("CH_CNM")]
        
        #uifscm = withChcnm[withChcnm.a.str.contains(app_name)]
        
        
        remAutoclicks = withChcnm[withChcnm.a.str.contains("em_monitoring") == False]
        

        
        if regionViewId != 'None':
            print("Region View chosen : ", regionViewId)
            prodSpecific = remAutoclicks[remAutoclicks.a.str.contains(regionViewId)]
        else:
            prodSpecific = remAutoclicks
        

        if clickidPattern != 'None':    
            print("clickIdPattern chosen:",clickidPattern)
            specificClickIds = prodSpecific[prodSpecific.a.str.contains(clickidPattern)]
        else:
            specificClickIds = prodSpecific

        log = specificClickIds
        
        if specificSession != 'None':
            log = log[log.a.str.contains(specificSession)]

                ####### Filter to process only given cluster's sesions #######################
        sessions_to_process = list()        
        if cluster != 'None' :       
            str_sessions_to_process= str((o.DSID.loc[o['Cluster'] == cluster]).tolist())
            str_sessions_to_process=str_sessions_to_process.replace("'","")
            sessions_to_process = str_sessions_to_process[1:-1].replace(" ","").split(',')
            print("Sessions to be processed : ", sessions_to_process)
        
        if len(sessions_to_process) != 0 :
#             criterion = lambda row: withspecificSession['DSID'] in sessions_to_process
#             w = withspecificSession[withspecificSession.apply(criterion, axis=1)]
            print("Data size before cluster filter : ", log.shape, " ..", "Applying filter of session: ", sessions_to_process)
            log1 = pd.DataFrame()
            for i,j in enumerate(sessions_to_process):
                log1 = log1.append(log[log.a.str.contains(j)])
            log = log1                     
            print("Data size after cluster filter : ", log.shape)       
            
            
        def getProcessedData(row):
            a = row.str.split('\] \[')[0]       
            for st in a:        
                b = st.split(':')
                if b[0].find('2019') != -1 : # to check for 2016 kind of..
                    TIME.append(str(b[0]+":"+b[1]+":"+b[2]).replace("[",''))            
                if("APP" == b[0]):
                    APP.append(b[1].strip())

#                 if("userId" == b[0]):
#                     userId.append(b[1].strip())

#                 if( "ecid" == b[0] ):
#                     ecid.append(b[1].strip()) #+':'+b[2].strip()

                if("DSID" == b[0]):
                    DSID.append(b[1].strip())

                if("CH_TTT" == b[0]):
                    CH_TTT.append(b[1].strip())

#                 if("CH_CET" == b[0]):
#                     CH_CET.append(b[1].strip())

#                 if("CH_CMP" == b[0]):
#                     CH_CMP.append(b[1].strip())

                if("CH_CNM" == b[0]):
                    CH_CNM.append(b[1].strip())

#                 if("CH_TYP" == b[0]):
#                     CH_TYP.append(b[1].strip())

#                 if("CH_WID" == b[0]):
#                     CH_WID.append(b[1].strip())

#                 if("CH_EID" == b[0]):
#                     CH_EID.append(b[1].strip().replace(']', ''))

                if("CH_RVD" == b[0]):
                    CH_RVD.append(b[1].strip().replace(']', ''))     

#                 if("CH_RTY" == b[0]):
#                     CH_RTY.append(b[1].strip().replace(']', ''))

#                 if("CH_CST" == b[0]):
#                     CH_CST.append(b[1].strip().replace(']', ''))

#                 if("CH_FAM" == b[0]):
#                     CH_FAM.append(b[1].strip().replace(']', ''))

                if("CH_CID" == b[0]):
                    b.pop(0) #remove "CH_CID"
                    x = ":".join(b)
                    CH_CID.append(x)

        #DSID,TIME,CH_CNM,CH_TYP,CH_TTT, CH_CET, CH_EID, CH_RTY, CH_CST, CH_CID, CH_CMP, CH_WID, CH_RVD, CH_FAM,APP, userId,ecid = ([] for i in range(17))
        DSID,TIME,CH_CNM,CH_CID,CH_RVD,APP,CH_TTT= ([] for i in range(7))
        
        
        
        for index, row in log.iterrows():
            getProcessedData(row)
        
        
        lists = [DSID,TIME,CH_CNM,CH_CID,CH_RVD,APP,CH_TTT]
        h = ['DSID','TIME','CH_CNM','CH_CID','CH_RVD','APP','CH_TTT']
        result2 = pd.concat([pd.Series(x) for x in lists], axis=1) 
        result2.columns = h
        # #### CUSTOM QUERIES ###################################################

        # 1. filter by item note/region view id
        # if regionViewId != 'None':
        #     result2 = result2.loc[result2['CH_RVD'].str.contains(regionViewId)]
        #result2 = result2.loc[result2['CH_CID'].str.contains("itemNode_projects_pjt_project_management")]
        #result2 = result2.loc[result2['CH_CID'].str.contains("itemNode_projects_projects")]


        # 2. Filter before/after a specific click 

        if interestedClick != 'None':   ####Interested clicks block starts##################  
            print("Interested click chosen : ", interestedClick)
            r = result2.index[result2.CH_CNM.str.contains(interestedClick)].values    
            #print("GOT INT CLICK ", r)
            stind = 0
            endind = len(result2)-1

            if len(r) != 0 : # If searched clicks is not found, then report everything
        #if interested click hppens more than once, take earliest one as stind and last occurence as endind
                 if type(r) == np.ndarray:
                    l = len(r)
                    stind = r[0]
                    endind = r[l-1]
                 else:
                    stind = r
                    enind = r

        #compute st end indcides based on delta selected
                 if stind-delta >= 0 :
                    stind = stind-delta
                 else:
                    stind = 0

                 if endind+delta <= len(result2)-1:
                    endind = endind+delta+1 #else endind as is
                 else:
                    endind = (len(result2))

                 result2 = result2[stind : endind]
        ####Interested clicks block ends##################


        # 3. Filter for specific user
        if userFilter != 'None':
            print("user filter chosen : ", userFilter.shape)
            result2 = result2.loc[result2['userId'].str.contains(userFilter)]


        # 4. Date filter
        if date != 'None':
            result2 = result2.loc[result2['TIME'].str.contains(date)]

        # 5. Click timings more than threshold
        # if totalTimeThreshold != 'None':
        #     result2 = result2.loc[np.int(result2['CH_TTT']) > totalTimeThreshold]

        #6. Specific Session Id
        
      
        

        # #### CUSTOM QUERIES ###################################################
        
        
        
        if (len(result2)) < 1 :
            print("After filters no data to process. Exiting ...")
            return
        else:
            sort_cols = ["DSID","TIME"] #Not sorting assuming we process file by file and in each file its already sorted
            f2 = result2[['DSID','TIME','CH_CID','CH_CNM','CH_TTT']]
            final_result = f2.sort_values(by=sort_cols, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
            for index, row in final_result.iterrows():              
                row['CH_CID'] = row['CH_CID'].replace(row['CH_CID'],re.sub(':*:', '',row['CH_CID'] ) )    


            # To hash the unique click ids
            col_list = final_result[['CH_CNM']] ## Hash based on chcid vs chcnm
            x= pd.DataFrame(final_result[['CH_CID','CH_CNM']])
            #col_list.dropna(axis=0,subset=['CH_CNM'],inplace=True)
            unique_col_list = col_list.drop_duplicates(keep='first')
            unique_col_list.insert(0, 'Key', range(0, len(unique_col_list)))

            d1 = pd.merge(unique_col_list,x,how='left',on=['CH_CNM'])

            nodedict = d1.set_index('Key')['CH_CNM'].to_dict()


            p_final = pd.merge(final_result,unique_col_list,how="left",on=['CH_CNM'])
            #print("NUMBER OF UNIQUE CLICKS FOUND ...",p_final.shape)
            outfile = open('ClickGraph.csv', 'w')
            p_final.to_csv(outfile)
            outfile.close()
            labels=pd.DataFrame(p_final.CH_CNM)
            df3 = pd.DataFrame(columns=['k1','k2','CH_CID'])
            df1 = pd.DataFrame(p_final.Key)
            df2 = pd.DataFrame(p_final.Key)
            df22 = pd.DataFrame(p_final.CH_CID)
            df3 = pd.concat([df1,df2,df22],axis=1)
            df3.columns= ['k1','k2','CH_CID']
            df4 = pd.concat([df3,labels],axis=1)

            def isNaN(num):
                return num != num

            #using graphviz
            if outputFileFolder == 'None':
                G = Digraph(format='jpeg')
            else:
                G = Digraph(format='jpeg',directory=outputFileFolder)
            G.attr(rankdir='LR')
            G.attr('node', shape='circle', width='1', nodesep='0.5')
            c = len(df4)
            nodelist = []
            Gdict = defaultdict(list)
            Gkeylabeldict = defaultdict(list)
            a = []

            for idx, row in df4.iterrows():
                curr_key = np.int(row.k1)    
                if isNaN(df4.k2.shift(-1)[idx]) == True:
                        next_key = df4.k2.shift(-1)[idx]
                else:
                    next_key = np.int(df4.k2.shift(-1)[idx])    
                curr_nodelabel=str(nodedict.get(row.CH_CID))

                #prepare node dictionary
                if isNaN(curr_key) == False: # 00 here it was next_key _21 nov
                    if curr_key in Gdict.keys():           
                        if type(Gdict[curr_key]) == list:            
                            #if next_key not in np.asarray(Gdict[curr_key]):                                
                                Gdict[curr_key].append(next_key)
                        elif type(Gdict[curr_key]) == np.int:            
                            #if next_key != Gdict[curr_key]:                                
                                Gdict[curr_key] = [Gdict[curr_key], next_key]
                    else:        
                        Gdict.update({curr_key:next_key})

                #capture labels of each key
                if curr_key not in Gkeylabeldict.keys():

                    o  = np.int(len(row.CH_CID))
                    Gkeylabeldict[curr_key] = str( row.CH_CID[-10:] )



            node1 = [None] * c
            node2 = [None] * c
            edgecount = [None] * c


            def find_element_in_list(element, list_element):
                try:
                    index_element = list_element.index(element)
                    return index_element
                except ValueError:
                    return -1


            def incrementEdgeCount(a,b,l1,l2,l3,d):
                a_index = find_element_in_list(a,l1)
                b_index = find_element_in_list(b,l2)
                edge_index_value = -1

                counter=0
                for i,j,k in zip(l1,l2,l3):
                    if (i == a) and (j == b) and (k != 0):
                        edge_index_value = k
                        edgecount[counter] = edgecount[counter]+1
                        break
                    counter = counter+1

                if edge_index_value == -1:
                    node1[d]=str(a)            
                    node2[d]=str(b)  
                    edgecount[d]=1
            #     if a_index != -1 and b_index != -1:
            #         edge_index_value = l3[a_index]

            #     if (a_index == b_index) and (a_index != -1) :                                
            #         edgecount[a_index] = l3[a_index] + 1
            #     else:        
            #         node1[d]=str(a)            
            #         node2[d]=str(b)  
            #         if edge_index_value != -1:            
            #             edgecount[d]=edge_index_value+1
            #         else:
            #             edgecount[d]=1


            # #Iterating thru dictionary and creating the graph 
            idx=0
            prevnode = -1
            l = len(Gdict)
            d=0

            #testing
            idx=0
            CreatedNodeCount=0

            nodeOrder=0
            for k,v in Gdict.items():    # NODE CREATION       

                #if isNaN(nodedict.get(k)) == False :
                    if isNaN(nodedict.get(k) ):        
                        nlabel = Gkeylabeldict.get(k)  # -- TO DO return back the CH_CID
                    else :
                        nlabel = nodedict.get(k)    
                    if idx == 0:
                        nodeOrder = nodeOrder+1
                        #print(nodeOrder,". ",nlabel)
                        G.node(str(k),label=str(nodeOrder)+"\n"+nlabel,color='Green',style='filled') #,fixedsize='true',width='2'        
                        CreatedNodeCount+=1
                    else:
                        #print(nodeOrder,". ",nlabel)
                        nodeOrder = nodeOrder+1
                        G.node(str(k),label=str(nodeOrder)+"\n"+nlabel,nodesize='10')            
                        CreatedNodeCount+=1
                    idx+=1

            print("NUMBER OF UNIQUE NODES CREATED : ", CreatedNodeCount)

            for k,v in Gdict.items():  # EDGE CREATION
                #print(k,"-->",v)
                if isNaN(k) == False:
                    if type(v) == list :        
                        if len(v) == 1 :                                            
                            incrementEdgeCount(str(k),str(np.int(v[0]) ),node1,node2,edgecount,d)
                            d=d+1
                        elif len(v) > 1 :
                            for i in v :
                                if (isNaN(str(k)) == False) and (isNaN(str(i)) == False or str(i).strip() != "nan") and (str(k) != str(i) ): # removing self-references                        
                                    incrementEdgeCount(str(k),str(i),node1,node2,edgecount,d)
                                    d=d+1
                    else:
                        if isNaN(v) == False :
                            #if (str(k) != str(np.int(v)) )  :     # removing self-references                           
                                incrementEdgeCount(str(k),str(np.int(v)),node1,node2,edgecount,d)
                                d=d+1

            highestEdge=0
            for i,j,k in zip(node1,node2,edgecount):
                if i != None and j != None and k != None :        
                    if highestEdge < k :
                        highestEdge = k

            # for i,j,k in zip(node1,node2,edgecount):
            #     print(i,"-->",j,"-->",k)
            for i,j,k in zip(node1,node2,edgecount):
                if i != None and j != None and k != None :        
                    #if k > 1:
                    if (k == highestEdge) and (highestEdge >= highlightEdgeCount):
                            G.edge(str(i),str(j),label=str((k)),color="Red",style="filled")
                    else:
                        print("drawing edge : ", str(i),"==>",str(j))
                        if (str(i) != 'nan' and str(j) != 'nan'):
                            G.edge(str(i),str(j),label=str((k)))

            G.render(output_graph_name,view=True)
            print("Graph creation completed ! ..", output_graph_name + ". Time taken : ",round(time.time() - start_time),"sec")

In [ ]:
# clickAnalyzer(inputFileFolder='D:\\ASTUDY\\AIML\\programs\\Fleet\\FlowLevel\\s\\UIServer_as1_03-clickhistory-9.log',
#               outputFileFolder='D:\\ASTUDY\\AIML\\programs\\Fleet\\FlowLevel\\s',
#               userSessionFile='D:\\ASTUDY\\AIML\\programs\\Fleet\\FlowLevel\\s\\output-usersessions.csv',
#               sessionOrCluster='0000MY13Men7q25prOs1yY1S91ZE000VK1')

import sys
clickAnalyzer(sys.argv[1],sys.argv[2],sys.argv[3],sys.argv[4])